In [6]:
import sys
import torch
from transformers import AutoTokenizer, AutoModel, pipeline, BertForMaskedLM, BertTokenizer, BertConfig, BertForPreTraining, BertTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from tokenizers import BertWordPieceTokenizer
    
sys.path.insert(0, '../')
from config import Config, PreTrainedType

In [7]:
model = BertForMaskedLM.from_pretrained(PreTrainedType.BertMultiLingual)
tokenizer = AutoTokenizer.from_pretrained(PreTrainedType.BertMultiLingual)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [97]:
example = '이순신은 조선 중기의 무신이다'
print(tokenizer.tokenize(example))
print(tokenizer(example))

['이', '##순', '##신은', '조선', '중', '##기의', '무신', '##이다']
{'input_ids': [2, 706, 1155, 7559, 2000, 754, 2605, 13160, 1895, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
# nlp_fill = pipeline('fill-mask', top_k=5, model=model, tokenizer=tokenizer)
# nlp_fill('Martin is living in [MASK].')

In [9]:
# !mkdir my_data

# # wiki corpu
# !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null
# !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt


# # korean corpus
# !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1_F5fziHjUM-jKr5Pwcx1we6g_J2o70kZ" > /dev/null
# !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1_F5fziHjUM-jKr5Pwcx1we6g_J2o70kZ" -o my_data/wiki_20190620.txt

In [23]:
# initialize empty tokenizer
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True, # whitespace 문자를 제거(\t, \n, \r, '')
    handle_chinese_chars=True,
    strip_accents=False, # [CamelCase] -> [Camel, Case]
    lowercase=False # Hello -> hello
)

In [24]:
# train
wp_tokenizer.train(
    files="my_data/wiki_20190620_small.txt",
    vocab_size=20000,   # vocab size 를 지정해줄 수 있습니다.
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##"
)

## Train BERT

In [25]:
tokenizer = BertTokenizer(
    vocab_file="./wordPieceTokenizer/my_tokenizer-vocab.txt",
    max_len=128,
    do_lower_case=False
)

In [26]:
tokenizer.add_special_tokens({'mask_token': '[MASK]'})
print(tokenizer.tokenize(example))

['이', '##순', '##신은', '조선', '중', '##기의', '무신', '##이다']


In [27]:
config = BertConfig(
    vocab_size=20000,
    hidden_size=512,
    num_hidden_layers=12,
    num_attention_heads=8,
    intermediate_size=3072, # transformer 내 FFN 사이즈
    hidden_act='gelu',
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    max_position_embeddings=128, # 최대 임베딩 사이즈. 최대 몇 개의 토큰까지를 이별긍로 받을지 결정. default. 512
    type_vocab_size=2, # default
    pad_token_id=0, # default
    position_embedding_type='absolute' # default
)

In [28]:
model = BertForPreTraining(config=config)
model.num_parameters()

61278754

## Data Collection for BERT

In [8]:
import os
import json
import pickle
import random
import time
import warnings
from filelock import FileLock
from typing import Dict, List, Optional
from torch.utils.data import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers.utils import logging

logger = logging.get_logger(__name__)

In [9]:
class TextDatasetForNextSentencePrediction(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        short_seq_probability=0.1,
        nsp_probability=0.5,
    ):
        # 여기 부분은 학습 데이터를 caching하는 부분입니다 :-)
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=True)
        self.short_seq_probability = short_seq_probability
        self.nsp_probability = nsp_probability

        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            directory,
            "cached_nsp_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        self.tokenizer = tokenizer

        lock_path = cached_features_file + ".lock"

        # Input file format:
        # (1) One sentence per line. These should ideally be actual sentences, not
        # entire paragraphs or arbitrary spans of text. (Because we use the
        # sentence boundaries for the "next sentence prediction" task).
        # (2) Blank lines between documents. Document boundaries are needed so
        # that the "next sentence prediction" task doesn't span between documents.
        #
        # Example:
        # I am very happy.
        # Here is the second sentence.
        #
        # A new document.

        with FileLock(lock_path):
            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )
            else:
                logger.info(f"Creating features from dataset file at {directory}")
                # 여기서부터 본격적으로 dataset을 만듭니다.
                self.documents = [[]]
                with open(file_path, encoding="utf-8") as f:
                    while True: # 일단 문장을 읽고
                        line = f.readline()
                        if not line:
                            break
                        line = line.strip()

                        # 이중 띄어쓰기가 발견된다면, 나왔던 문장들을 모아 하나의 문서로 묶어버립니다.
                        # 즉, 문단 단위로 데이터를 저장합니다.
                        if not line and len(self.documents[-1]) != 0:
                            self.documents.append([])
                        tokens = tokenizer.tokenize(line)
                        tokens = tokenizer.convert_tokens_to_ids(tokens)
                        if tokens:
                            self.documents[-1].append(tokens)
                # 이제 코퍼스 전체를 읽고, 문서 데이터를 생성했습니다! :-)
                logger.info(f"Creating examples from {len(self.documents)} documents.")
                self.examples = []
                # 본격적으로 학습을 위한 데이터로 변형시켜볼까요?
                for doc_index, document in enumerate(self.documents):
                    self.create_examples_from_document(document, doc_index) # 함수로 가봅시다.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def create_examples_from_document(self, document: List[List[int]], doc_index: int):
        """Creates examples for a single document."""
        # 문장의 앞, 뒤에 [CLS], [SEP] token이 부착되기 때문에, 내가 지정한 size에서 2 만큼 빼줍니다.
        # 예를 들어 128 token 만큼만 학습 가능한 model을 선언했다면, 학습 데이터로부터는 최대 126 token만 가져오게 됩니다.
        max_num_tokens = self.block_size - self.tokenizer.num_special_tokens_to_add(pair=True)

        # We *usually* want to fill up the entire sequence since we are padding
        # to `block_size` anyways, so short sequences are generally wasted
        # computation. However, we *sometimes*
        # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
        # sequences to minimize the mismatch between pretraining and fine-tuning.
        # The `target_seq_length` is just a rough target however, whereas
        # `block_size` is a hard limit.

        # 여기가 재밌는 부분인데요!
        # 위에서 설명했듯이, 학습 데이터는 126 token(128-2)을 채워서 만들어지는게 목적입니다.
        # 하지만 나중에 BERT를 사용할 때, 126 token 이내의 짧은 문장을 테스트하는 경우도 분명 많을 것입니다 :-)
        # 그래서 short_seq_probability 만큼의 데이터에서는 2-126 사이의 random 값으로 학습 데이터를 만들게 됩니다.
        target_seq_length = max_num_tokens
        if random.random() < self.short_seq_probability:
            target_seq_length = random.randint(2, max_num_tokens)

        current_chunk = []  # a buffer stored current working segments
        current_length = 0
        i = 0

        # 데이터 구축의 단위는 document 입니다
        # 이 때, 무조건 문장_1[SEP]문장_2 이렇게 만들어지는 것이 아니라,
        # 126 token을 꽉 채울 수 있게 문장_1+문장_2[SEP]문장_3+문장_4 형태로 만들어질 수 있습니다.
        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    # `a_end` is how many segments from `current_chunk` go into the `A`
                    # (first) sentence.
                    a_end = 1
                    # 여기서 문장_1+문장_2 가 이루어졌을 때, 길이를 random하게 짤라버립니다 :-)
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)
                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])
                    # 이제 [SEP] 뒷 부분인 segmentB를 살펴볼까요?
                    tokens_b = []
                    # 50%의 확률로 랜덤하게 다른 문장을 선택하거나, 다음 문장을 학습데이터로 만듭니다.
                    if len(current_chunk) == 1 or random.random() < self.nsp_probability:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        # This should rarely go for more than one iteration for large
                        # corpora. However, just to be careful, we try to make sure that
                        # the random document is not the same as the document
                        # we're processing.
                        for _ in range(10):
                            random_document_index = random.randint(0, len(self.documents) - 1)
                            if random_document_index != doc_index:
                                break
                        # 여기서 랜덤하게 선택합니다 :-)
                        random_document = self.documents[random_document_index]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        # We didn't actually use these segments so we "put them back" so
                        # they don't go to waste.
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    # Actual next
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    # 이제 126 token을 넘는다면 truncation을 해야합니다.
                    # 이 때, 126 token 이내로 들어온다면 행위를 멈추고,
                    # 만약 126 token을 넘는다면, segmentA와 segmentB에서 랜덤하게 하나씩 제거합니다.
                    def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
                        """Truncates a pair of sequences to a maximum sequence length."""
                        while True:
                            total_length = len(tokens_a) + len(tokens_b)
                            if total_length <= max_num_tokens:
                                break
                            trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
                            assert len(trunc_tokens) >= 1
                            # We want to sometimes truncate from the front and sometimes from the
                            # back to add more randomness and avoid biases.
                            if random.random() < 0.5:
                                del trunc_tokens[0]
                            else:
                                trunc_tokens.pop()

                    truncate_seq_pair(tokens_a, tokens_b, max_num_tokens)

                    assert len(tokens_a) >= 1
                    assert len(tokens_b) >= 1

                    # add special tokens
                    input_ids = self.tokenizer.build_inputs_with_special_tokens(tokens_a, tokens_b)
                    # add token type ids, 0 for sentence a, 1 for sentence b
                    token_type_ids = self.tokenizer.create_token_type_ids_from_sequences(tokens_a, tokens_b)
                    
                    # 드디어 아래 항목에 대한 데이터셋이 만들어졌습니다! :-)
                    # 즉, segmentA[SEP]segmentB, [0, 0, .., 0, 1, 1, ..., 1], NSP 데이터가 만들어진 것입니다 :-)
                    # 그럼 다음은.. 이 데이터에 [MASK] 를 씌워야겠죠?
                    example = {
                        "input_ids": torch.tensor(input_ids, dtype=torch.long),
                        "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                        "next_sentence_label": torch.tensor(1 if is_random_next else 0, dtype=torch.long),
                    }

                    self.examples.append(example)

                current_chunk = []
                current_length = 0

            i += 1

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

In [10]:
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=tokenizer,
    file_path='./my_data/wiki_20190620_small.txt',
    block_size=128,
    overwrite_cache=False,
    short_seq_probability=0.1,
    nsp_probability=0.5,
)

data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [12]:
dataset[0]

{'input_ids': tensor([   101,   9672,  36240,  12605,   9551,    107,   9706,  22458,    107,
           9786,  21876,   9689,  25503, 106340,   9311,  16323,  21928,   9768,
          25387,  23545,  11303,  48506,  70672,  30919,    119,   9706,  22458,
           9786,  21876,  11018,   9678,  12508,  16985,  16323,   9430,  21876,
           9786,  21614,  45725,   9944,  56645,  12030,  12605,   9246,  10622,
          11489,  88921,    119,    102,  32537,  78686,   9641,  12609,    119,
          11087,  10954,  23545,   9960,  17360,   9069,  92454,   9576,  50450,
          36251,  18347,   9136, 119312,    217,   9279,  18227,  33727,   8843,
         118698,  25685,   9089,  10622,   9339,  17706,    119,  21555,   9353,
          66923,    107,   9136, 119312,   9027,  14646,    107,   9202,   9524,
          26737,  32855,    119,  10828,  10954,   9678,  12508,  16985,   9689,
           9414,  14279,   9637,  14279,   9428,  41521,  11489,   8983,  18471,
          35506

In [13]:
for example in dataset.examples[0:1]:
    print(example)

{'input_ids': tensor([   101,   9672,  36240,  12605,   9551,    107,   9706,  22458,    107,
          9786,  21876,   9689,  25503, 106340,   9311,  16323,  21928,   9768,
         25387,  23545,  11303,  48506,  70672,  30919,    119,   9706,  22458,
          9786,  21876,  11018,   9678,  12508,  16985,  16323,   9430,  21876,
          9786,  21614,  45725,   9944,  56645,  12030,  12605,   9246,  10622,
         11489,  88921,    119,    102,  32537,  78686,   9641,  12609,    119,
         11087,  10954,  23545,   9960,  17360,   9069,  92454,   9576,  50450,
         36251,  18347,   9136, 119312,    217,   9279,  18227,  33727,   8843,
        118698,  25685,   9089,  10622,   9339,  17706,    119,  21555,   9353,
         66923,    107,   9136, 119312,   9027,  14646,    107,   9202,   9524,
         26737,  32855,    119,  10828,  10954,   9678,  12508,  16985,   9689,
          9414,  14279,   9637,  14279,   9428,  41521,  11489,   8983,  18471,
         35506,  16439,   

In [14]:
print(data_collator(dataset.examples))

{'input_ids': tensor([[  101,  9672, 36240,  ..., 17594, 37909,   102],
        [  101, 32537, 12490,  ..., 12490,   119,   102],
        [  101, 21890, 70347,  ..., 10003, 30005,   102],
        ...,
        [  101,   103,  8885,  ...,   103,  9694,   102],
        [  101,   103,   117,  ...,  8932, 21611,   102],
        [  101,  9272, 22333,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]]), 'next_sentence_label': tensor([0, 0, 0,  ..., 1, 0, 0]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  

In [49]:
print(data_collator(dataset.examples)['labels'][0]) # -100이 뭐지? 아무런 변형하지 않은 부분인 것 같긴 하다

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  5379,  -100,  -100,  2407,    16,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  3666,
           16,  -100,  -100,  -100,  -100,  -100,  -100,   174,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
           94,   438,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100, 17663,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  1212,  -100,  4860,  -100,  -100,   174,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  6530,  -100,
         -100,  -100,  -100,  -100,    16,  -100,  -100,  -100,  -100,  -100,
         -100, 17629,  -100,  -100,  -100])


In [76]:
sample = data_collator(dataset.examples)
encode_sample = sample['input_ids'][0].tolist()
label_sample = sample['labels'][0].tolist()
print(tokenizer.decode(encode_sample))

[CLS] 제임스 얼 [MASK] 지미 " 카터 주니 [MASK] 민주당 출신 미국 39번째 대통령 이다. 지미 카터는 [MASK] 섬터 카 [MASK]티 플레인스 마을에 [MASK] 태어났다. 조지아 공과대학교를 졸업하였다 [MASK] [MASK] 후 해군 [MASK] 들어가 전 [MASK] · 원자력 · 잠수함의 승무 총 일하였다 [MASK] [SEP] [MASK] 예편하였고 이후 땅콩 · [MASK]화 등을 [UNK] 많은 돈을 벌었다. 그의 별명이 " 땅콩 농부 " [MASK] 알려졌다. [MASK] 조지아 [MASK] [MASK]원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 [MASK] 낙선하지만 1970년 조지아 주 지사를 [MASK]했다. 대통령이 [MASK] 전 조지아주 상원의원을 [MASK] 연임했으며 [SEP]


In [90]:
idx = 19
print('Input ID', [encode_sample[idx]])
print('Label', [label_sample[idx]])
print('Decoded', tokenizer.decode([encode_sample[idx]]))

Input ID [4]
Label [16249]
Decoded [MASK]


Mask Input ID: 4  
Mask Label: 5, 2489, 16249 <- 해당 위치에 원래 어떤 레이블이 있었는지. -100읽 경우에는 replace 또는 masking이 되지 않은 부분임

In [67]:
tokenizer.decode(data_collator(dataset.examples)['input_ids'][0].tolist())

'[CLS] [MASK] 얼 " 지미 " 카터 주니어는 민주당 출신 미국 39번째 대통령 이다 [MASK] 지미 [MASK] 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함 · 원자력 · 잠수함의 승무원으로 일하였다 [MASK] [SEP]위로 [MASK]편하였고 이후 땅콩 · 면화 등을 [UNK] 많은 돈을 벌었다. 그의 별명이 " 땅콩 농부 [MASK] 로 알려졌다. 1962년 조지아 [MASK] 상원 의원 [MASK] 낙선하나 그 [MASK] [MASK] DN음을 입증럭 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 [MASK] 조지아 주 지사를 역임 [MASK]. 대통령이 [MASK] 전 [MASK] 상원의원을 두번 연임했으며 [SEP]'

## Trainer를 활용한 학습

In [96]:
training_args = TrainingArguments(
    output_dir='model_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    save_steps=1000, # 1000 스텝마다 모델을 저장하겠다
    save_total_limit=2, # 마지막 두 스텝 iteration에 대한 모델만 저장하고 나머지는 삭제하겠다. 지나치게 많은 모델이 저장되지 않도록 하기 위해 사용
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [95]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
100,8.430500
200,7.695400
300,7.529500
400,7.471800
500,7.445700
600,7.268400
700,7.211400
800,7.186700


In [ ]:
t

In [98]:
tokenizer.tokenize(example)

['이', '##순', '##신은', '조선', '중', '##기의', '무신', '##이다']

In [ ]:
nlp_fill = pipeline('fill-mask', top_k=5, model=m)